# 3.

In [49]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
import time
import math
import pandas as pd
import pickle
from tqdm import tqdm

In [28]:
df_products = pickle.load(open('danawa_crawling_airpurifier_result_240607.pk1','rb'))
df_products.head()

,제품모델명,출시시기,리뷰평점,리뷰개수,헤드메시지,스펙,링크 url,가격정보_1,최저가_1,가격정보_2,최저가_2,가격정보_3,최저가_3
0,LG전자 퓨리케어 360˚ 플러스 AS303DWFA,2023.01.,4.8,152,이제 플러스 모델에도 UP가전으로 업그레이드 가능!,공기청정기 / 30.3평(100㎡) / 소비전력: 70W / 버튼잠금 / 풍량조절 ...,https://prod.danawa.com/info/?pcode=18775037&c...,일반구매,"736,090원",무빙휠 포함,"856,970원",,
1,삼성전자 블루스카이 3100 AX033B310GGD,2022.03.,4.7,"1,065",,공기청정기 / 10평(33㎡) / 소비전력: 26W / 버튼잠금 / 풍량조절 / 스...,https://prod.danawa.com/info/?pcode=16536392&c...,1개,"153,000원",2개,"382,700원",추가필터 포함,"269,000원"
2,LG전자 오브제컬렉션 퓨리케어 360˚ UV살균 AS354NS3A,2024.02.,4.9,69,LG전자 24년형 PA인증받은 UV살균 공기청정기 출시!,공기청정기 / 34평(114㎡) / 소비전력: 80W / 버튼잠금 / 풍량조절 / ...,https://prod.danawa.com/info/?pcode=34738715&c...,일반구매,"1,311,990원",무빙휠 포함,"1,423,760원",,
3,LG전자 퓨리케어 360˚ Hit AS153HWWC,2023.03.,4.8,678,퓨리케어360˚ 1단 모델 대비 16% 슬림해진 에어라운드 디자인 적용,공기청정기 / 15평(50㎡) / 소비전력: 34W / 버튼잠금 / 풍량조절 / 스...,https://prod.danawa.com/info/?pcode=19312541&c...,,"311,380원",,,,
4,삼성전자 블루스카이 5500 AX060CG500GGD,2023.01.,4.6,91,실내 공기질까지 학습해 맞춰주는 AI 청정기능 적용,공기청정기 / 18평(60㎡) / 소비전력: 60W / 버튼잠금 / 풍량조절 / 이...,https://prod.danawa.com/info/?pcode=18852014&c...,일반구매,"247,690원",추가필터 포함,"419,000원",,


In [29]:
df_products['리뷰개수'] = df_products['리뷰개수'].str.replace(',', '')

In [30]:
df_products['리뷰개수'] = pd.to_numeric(df_products['리뷰개수'], errors='coerce'); df_products['리뷰개수']

0       152.0
1      1065.0
2        69.0
3       678.0
4        91.0
        ...  
396      29.0
397      71.0
398      89.0
399      88.0
400       NaN
Name: 리뷰개수, Length: 401, dtype: float64

In [31]:
df_for_review_crawling = df_products[df_products['리뷰개수'] >= 10]
len(df_for_review_crawling)

244

In [34]:
#상위 10개 제품만 선정, 데이터프레임 인덱스 초기화
df_for_review_crawling = df_for_review_crawling[:10]

df_for_review_crawling = df_for_review_crawling.reset_index()
df_for_review_crawling

,index,제품모델명,출시시기,리뷰평점,리뷰개수,헤드메시지,스펙,링크 url,가격정보_1,최저가_1,가격정보_2,최저가_2,가격정보_3,최저가_3
0,0,LG전자 퓨리케어 360˚ 플러스 AS303DWFA,2023.01.,4.8,152.0,이제 플러스 모델에도 UP가전으로 업그레이드 가능!,공기청정기 / 30.3평(100㎡) / 소비전력: 70W / 버튼잠금 / 풍량조절 ...,https://prod.danawa.com/info/?pcode=18775037&c...,일반구매,"736,090원",무빙휠 포함,"856,970원",,
1,1,삼성전자 블루스카이 3100 AX033B310GGD,2022.03.,4.7,1065.0,,공기청정기 / 10평(33㎡) / 소비전력: 26W / 버튼잠금 / 풍량조절 / 스...,https://prod.danawa.com/info/?pcode=16536392&c...,1개,"153,000원",2개,"382,700원",추가필터 포함,"269,000원"
2,2,LG전자 오브제컬렉션 퓨리케어 360˚ UV살균 AS354NS3A,2024.02.,4.9,69.0,LG전자 24년형 PA인증받은 UV살균 공기청정기 출시!,공기청정기 / 34평(114㎡) / 소비전력: 80W / 버튼잠금 / 풍량조절 / ...,https://prod.danawa.com/info/?pcode=34738715&c...,일반구매,"1,311,990원",무빙휠 포함,"1,423,760원",,
3,3,LG전자 퓨리케어 360˚ Hit AS153HWWC,2023.03.,4.8,678.0,퓨리케어360˚ 1단 모델 대비 16% 슬림해진 에어라운드 디자인 적용,공기청정기 / 15평(50㎡) / 소비전력: 34W / 버튼잠금 / 풍량조절 / 스...,https://prod.danawa.com/info/?pcode=19312541&c...,,"311,380원",,,,
4,4,삼성전자 블루스카이 5500 AX060CG500GGD,2023.01.,4.6,91.0,실내 공기질까지 학습해 맞춰주는 AI 청정기능 적용,공기청정기 / 18평(60㎡) / 소비전력: 60W / 버튼잠금 / 풍량조절 / 이...,https://prod.danawa.com/info/?pcode=18852014&c...,일반구매,"247,690원",추가필터 포함,"419,000원",,
5,5,위닉스 타워 프라임 APRM833-JWK,2019.11.,4.8,3349.0,위닉스가 준비한 프리미엄 공기청정기!,공기청정기 / 25.9평(85.8㎡) / 소비전력: 47W / 풍량조절 / 이동식바...,https://prod.danawa.com/info/?pcode=9925041&ca...,일반구매,"289,070원",,,,
6,6,LG전자 퓨리케어 360˚ Hit AS183HWWA,2023.03.,4.8,172.0,퓨리케어360˚ 1단 모델 대비 16% 슬림해진 에어라운드 디자인 적용,공기청정기 / 18.7평(62㎡) / 소비전력: 41W / 버튼잠금 / 풍량조절 /...,https://prod.danawa.com/info/?pcode=19312682&c...,일반구매,"356,450원",,,,
7,7,삼성전자 블루스카이 3100 AX033B310GBD,2022.03.,4.7,222.0,,공기청정기 / 10평(33㎡) / 소비전력: 26W / 버튼잠금 / 풍량조절 / 스...,https://prod.danawa.com/info/?pcode=16536521&c...,일반구매,"148,990원",추가필터 포함,"199,000원",,
8,8,삼성전자 비스포크 큐브 Air 인피니트 라인 AX033DB900EGD,2024.03.,4.7,14.0,24년형 4way 서라운드 청정+강력 탈취 S필터 탑재 Infinite Line 공...,공기청정기 / 10평(33㎡) / 소비전력: 15W / 풍량조절 / 스마트기능: 삼...,https://prod.danawa.com/info/?pcode=39553769&c...,1개,"522,450원",2개,"1,299,000원",,
9,9,LG전자 퓨리케어 360˚ 플러스 AS193DWFA,2023.01.,4.8,151.0,이제 플러스 모델에도 UP가전으로 업그레이드 가능!,공기청정기 / 18.5평(61.2㎡) / 소비전력: 48W / 버튼잠금 / 풍량조절...,https://prod.danawa.com/info/?pcode=18774986&c...,일반구매,"602,330원",무빙휠 포함,"735,640원",,


# 3-1 [함수1] 크롬드라이버 실행부터 페이지 넘기면서 크롤링

In [35]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
import time
import math
import pandas as pd
import pickle

In [39]:
def crawling_danawa_reviews_get_sources(df_for_review_crawling):
        #함수 입력값: 제품정보 데이터프레임
    
    ### 드라이버 실행 ~ 타겟 제조사 선택까지 ###
    my_options = webdriver.ChromeOptions()
    my_options.add_argument("headless")
    my_options.add_argument("--window-size=1920x1080")
    my_options.add_argument("disable-gpu")
    driver = webdriver.Chrome(options=my_options)
    driver.implicitly_wait(1)
    
    #크롤링한 페이지소스 저장할 빈 딕셔너리 생성
    result_dict = {}
    
    ### 제품별로 돌아가면서 상세페이지 접속 ###
    
    for u in range(len(df_for_review_crawling)):
        driver.get(df_for_review_crawling['링크 url'][u])
        print(u+1, ".", df_for_review_crawling['제품모델명'][u], end = ' ')
        time.sleep(3)
        
        #제품별로 저장할 빈 딕셔너리 생성
        soup_dict = {}
        
        #쇼핑몰 상품리뷰 개수 가져오기
        soup = BeautifulSoup(driver.page_source)
        num_reviews = soup.select_one('div.point_num span strong').text.strip().replace(',','')
        print('-', num_reviews, '개')
        
        #리뷰 총 몇 페이지인지 계산
        num_pages = math.ceil(int(num_reviews) / 10)
        
        
        ### 리뷰페이지 넘기면서 페이지 소스 크롤링 및 저장 ###
        #page에 1씩 더하면서, 전체 페이지 수와 같아지기 전까지 반복
        page = 1
        while page < num_pages:
            
            #페이지별 리뷰 전부 가져오기
            soup = BeautifulSoup(driver.page_source)
            reviews = soup.select('ul.rvw_list li.danawa-prodBlog-companyReview-clazz-more')
            
            #페이지별 리뷰 딕셔너리에 저장해두기
            soup_dict[page] = reviews
            
            page += 1
            
            #페이지 넘기는 부분 경로 통째로 가져와 놓고
            page_nums = driver.find_element(By.XPATH, '//*[@id="danawa-prodBlog-companyReview-content-list"]/div/div')
            
            
            #11,21,31 번째 페이지인 경우 [다음 페이지] 버튼 클릭
            if page % 10 == 1:
                page_nums.find_element(By.CLASS_NAME, 'point_arw_r').click()
                
            
            #나머지 경우는 page에 해당하는 페이지 클릭
            else:
                page_nums.find_element(By.LINK_TEXT, str(page)).click()
                
            
            #에러나지 않도록 한 페이지마다 잠시 대기
            print('.', end = ' ')
            time.sleep(2)
            
            
        #마지막 페이지 리뷰 가져오고 마무리
        soup = BeautifulSoup(driver.page_source)
        reviews = soup.select('ul.rvw_list li.danawa-prodBlog-companyReview-clazz-more')
        soup_dict[page] = reviews
        print("완료")
        
        result_dict[u+1] = soup_dict
        
    
    #함수 출력값: 제품별 리뷰페이지 소스 저장한 딕셔너리
    return result_dict

In [40]:
result_dict = crawling_danawa_reviews_get_sources(df_for_review_crawling)

1 . LG전자 퓨리케어 360˚ 플러스 AS303DWFA - 151 개
. . . . . . . . . . . . . . . 완료
2 . 삼성전자 블루스카이 3100 AX033B310GGD - 1063 개
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 완료
3 . LG전자 오브제컬렉션 퓨리케어 360˚ UV살균 AS354NS3A - 70 개
. . . . . . 완료
4 . LG전자 퓨리케어 360˚ Hit AS153HWWC - 680 개
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 완료
5 . 삼성전자 블루스카이 5500 AX060CG500GGD - 91 개
. . . . . . . . . 완료
6 . 위닉스 타워 프라임 APRM833-JWK - 3354 개
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

# 3-2. [함수2] 수집된 정보 거르고 필터링해서 데이터 프레임으로

In [47]:
def crawling_danawa_reviews_result(result_dict, df_for_review_crawling, excel_title):
    #함수 입력값: 리뷰소스 저장된 딕셔너리, 제품정보 데이터프레임, 결과 저장할 엑셀파일명
    
    #제품명 풀네임, 제조사, 모델명
    name_list, brand_list, model_list = [], [], []
    #별점, 구매처, 작성일, 리뷰내용
    star_list, mall_list, date_list, review_list = [], [], [], []
    
    #제품 하나씩 돌리기
    for key_p in tqdm(range(len(result_dict))):
        soup_dict = result_dict[key_p+1]
        
        
        #제품명 풀네임, 제조사, 모델명 가져오기
        name = df_for_review_crawling['제품모델명'][key_p]
        brand = name.split()[0]
        model = name.split()[-1]
        
        #저장해둔 리뷰페이지 하나씩 돌리기
        for key_r in range(len(soup_dict)):
            reviews = soup_dict[key_r+1]
            
            #리뷰마다 돌리면서 내용 가져오기
            for i in range(len(reviews)):
                
                #가져와둔 제품명 풀네임, 제조사, 모델명
                name_list.append(str(name))
                brand_list.append(str(brand))
                model_list.append(str(model))
                
                
                #별점        
                try:
                    star_list.append(int
                            (reviews[i].select_one('span.star_mask').text.strip().replace('점',''))/20)
                except:
                    star_list.append('')
            
                #구매처
                try:
                    mall_list.append(str(reviews[i].select_one('span.mall').text.strip()))
                except:
                    mall_list.append('')
        
                #작성일
                try:
                    date_list.append(str(reviews[i].select_one('span.date').text.strip()))
                except:
                    date_list.append('')            

                #리뷰내용
                try:
                    review_list.append(str(reviews[i].select_one('div.atc').text.strip()))
                except:
                    review_list.append('')
                    
    ### 크롤링 결과 저장 ###
    #데이터 프레임 생성
    df_result = pd.DataFrame({
        '제조사':brand_list, '모델명':model_list, '제품명':name_list,
        '구매처':mall_list, '작성일': date_list, '별점': star_list, '리뷰':review_list
    })
    
    #데이터프레임을 엑셀파일로 저장
    with pd.ExcelWriter(excel_title) as writer:
        df_result.to_excel(writer)
        
    return df_result

In [50]:
df_result = crawling_danawa_reviews_result(result_dict, df_for_review_crawling, 'danawa_crawling_airpurifier_reviews_result_240610.xlsx')

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:04<00:00,  2.05it/s]


In [51]:
df_result

,제조사,모델명,제품명,구매처,작성일,별점,리뷰
0,LG전자,AS303DWFA,LG전자 퓨리케어 360˚ 플러스 AS303DWFA,엘지이닷컴,2024.04.30.,5.0,오랫동안 갖고 싶었던 제품입니다. 일단 디자인 너무 예쁘고 헤드 부분이 공기질에 따...
1,LG전자,AS303DWFA,LG전자 퓨리케어 360˚ 플러스 AS303DWFA,엘지이닷컴,2024.04.24.,5.0,퓨리케어 정말좋네요~ 많이들 사용하는이유를\n알겠더라구요.소음도 거의없고~ 알아서 ...
2,LG전자,AS303DWFA,LG전자 퓨리케어 360˚ 플러스 AS303DWFA,엘지이닷컴,2024.06.08.,5.0,집안 미세먼지 걱정에 구입을 했습니다. 구입후 미세먼지 뿐만 아니라 냄새까지 잡아주...
3,LG전자,AS303DWFA,LG전자 퓨리케어 360˚ 플러스 AS303DWFA,엘지이닷컴,2024.04.15.,5.0,자동 공기청정 모드로 해놓으면 알아서 공기청정 해줍니다.\n공기가 상쾌해지는 기분이에요
4,LG전자,AS303DWFA,LG전자 퓨리케어 360˚ 플러스 AS303DWFA,엘지이닷컴,2024.06.03.,5.0,강아지가 있고 미세먼지 등 요즘 너무 걱정이여서 구매한 제품이에요~~ 구독으로 했으...
...,...,...,...,...,...,...,...
5963,LG전자,AS193DWFA,LG전자 퓨리케어 360˚ 플러스 AS193DWFA,G마켓,2023.12.13.,3.0,배송비는 바로 받아놓고 계속 전화도 안받으셨지만... 배송믄 2주 조금 더 걸렸습니...
5964,LG전자,AS193DWFA,LG전자 퓨리케어 360˚ 플러스 AS193DWFA,옥션,2024.03.13.,5.0,배송도 빠르고 친절하였습니다
5965,LG전자,AS193DWFA,LG전자 퓨리케어 360˚ 플러스 AS193DWFA,옥션,2024.01.29.,5.0,공기청정기는 lg제품이 최고입니다적당한가격에 구매하여 아껴가며 오래도록 쓰겠 습니다
5966,LG전자,AS193DWFA,LG전자 퓨리케어 360˚ 플러스 AS193DWFA,SSG.COM,2024.06.02.,5.0,배송도 지정일에 잘왔고 싸게 잘 샀어요
